# Initial Setting

In [1]:
import nest_asyncio
nest_asyncio.apply()

import numpy as np
import tensorflow as tf
import random

FRACTION=0.1
BATCH_SIZE = 10 # inf = -1
NUM_EPOCHS = 5 # fixed!
TRAINING_ROUNDS=18

CLIENTS_SHUFFLE_PER_ROUND=False
#CLIENTS_SHUFFLE_PER_ROUND=True 

# AutoEncoder for reducing parameter number

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

class Autoencoder(Model):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Flatten(), # make origianl 2-dim to 1-dim
      layers.Dense(26*16, activation='relu'),
      layers.Dense(26*8, activation='relu'),
      layers.Dense(26*4, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      # input dim is 64
      layers.Dense(26*8, activation='relu'),
      layers.Dense(26*16, activation='relu'),
      layers.Dense(832, activation='relu'), # It match the encode-flatten()
      layers.Reshape((26, 32)) # restore 1-dim to 2-dim
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder()

In [4]:
autoencoder.load_weights("/content/drive/MyDrive/Federated-Learning/ae_1000_exam_weights.ckpt")

# Make Preprocessed-(I.I.D)Dataset

In [5]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data() # This dataset is not "E"mnist. Don't confuse!

raw_dataset_for_iid=list(zip(mnist_train[0].reshape(-1, 28, 28, 1).astype("float32")/255.0, mnist_train[1].astype("float32")))
random.shuffle(raw_dataset_for_iid)

el_size=600
temp_list_for_image=[]
temp_list_for_label=[]
federated_train_data_for_iid=[]
for idx, el in enumerate(raw_dataset_for_iid) :
    temp_list_for_image.append(el[0])
    temp_list_for_label.append(el[1])
    if (idx+1)%(el_size)==0 :
        federated_train_data_for_iid.append((np.array(temp_list_for_image, dtype="float32"), np.array(temp_list_for_label, dtype="float32")))
        temp_list_for_image=[]
        temp_list_for_label=[]
        
federated_train_data = federated_train_data_for_iid

# Make MNIST-CNN 99% model using Keras

In [6]:
keras_model= tf.keras.models.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Conv2D(64, kernel_size=(5, 5), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation="softmax"),
])

keras_model.summary()

keras_model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

# Start Training

In [7]:
def encode_concat_parameter(parameter) :
    layer1 = parameter[0].reshape(25,32)
    lyaer2 = parameter[1].reshape(1,32)
    concat_L1_L2 = np.array([np.concatenate((layer1,lyaer2))])
    encode_concat_L1_L2 = autoencoder.encoder(concat_L1_L2).numpy()
    temp_list=[encode_concat_L1_L2]
    for i in range(2,8) :
      temp_list.append(parameter[i])
    return temp_list

def decode_seperate_parameter(parameter) :
    encoded_concat_L1_L2=autoencoder.decoder(parameter[0]).numpy()
    concat_L1_L2=encoded_concat_L1_L2[0]    
    L1_temp = concat_L1_L2[0:25]
    L1_reshape = L1_temp.reshape(5,5,1,32)
    L2_temp = concat_L1_L2[25]
    L2_reshape = L2_temp.reshape(32,)
    temp_list=[L1_reshape, L2_reshape]
    for i in range(1, 7) :
      temp_list.append(parameter[i])
    return temp_list
    

In [8]:
TOTAL_CLIENTS = len(federated_train_data)
SELECTED_CLIENTS = int(TOTAL_CLIENTS*FRACTION)
print("total client :", TOTAL_CLIENTS, ", selected client :", SELECTED_CLIENTS)

# starting to training
selected_clients_list=clients_status_list=np.random.choice(TOTAL_CLIENTS, size=SELECTED_CLIENTS, replace=False) # that is relevant to 4-2 step.

global_parameter=keras_model.get_weights() # actually, It is initial parameter...
encoded_global_parameter=encode_concat_parameter(global_parameter) #### encoder

#import sys
#sys.exit()

print("-- prameter shape --")
for layer in global_parameter :
    print(layer.shape)

list_of_local_parameter=[]
list_of_local_dataset_size=[]
list_of_local_accuracy=[]
list_of_local_loss=[]

for round in range(TRAINING_ROUNDS) :
    print("\n▶ Round", round+1, "◀")
    
        # check whether to apply shuffle mode per round
    if CLIENTS_SHUFFLE_PER_ROUND == True :
        selected_clients_list = np.random.choice(TOTAL_CLIENTS, size=SELECTED_CLIENTS, replace=False)
    #print("selected clients :", selected_clients_list)

        # recevie Local parameter.
    for client_dataset in selected_clients_list :
        train_images, train_labels=federated_train_data[client_dataset]
        
        decoded_global_parameter = decode_seperate_parameter(encoded_global_parameter)#### decoder

        keras_model.set_weights(decoded_global_parameter)
        
        train_result=keras_model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
            
        local_parameter=keras_model.get_weights()
        encoded_local_parameter= encode_concat_parameter(local_parameter) #### encoder
        list_of_local_parameter.append(encoded_local_parameter)
        list_of_local_dataset_size.append(len(train_images))
        list_of_local_accuracy.append(train_result.history["accuracy"][-1])
        list_of_local_loss.append(train_result.history["loss"][-1])
        
        #print("    clint ID :", client_dataset, "training complete.")
        #print("        accuracy :", train_result.history["accuracy"][-1], "- loss :", train_result.history["loss"][-1])
    
        #4-5. aggregate Local parameters.
    decoded_list_of_loca_parameter=[ decode_seperate_parameter(encoded_local_parameter) for encoded_local_parameter in list_of_local_parameter] #### decoder
    global_parameter = np.mean(decoded_list_of_loca_parameter, axis=0)
    encoded_global_parameter=encode_concat_parameter(global_parameter) #### encoder

    #global_parameter = np.mean(list_of_local_parameter, axis=0)*np.sum)(list_of_local_dataset_size
    #print("global_parameter :",global_parameter)
    current_mean_accuracy = np.mean(np.array(list_of_local_accuracy, dtype="float32"))
    current_mean_loss = np.mean(np.array(list_of_local_loss, dtype="float32"))
    print(f"  evaluation mean : accuracy - {current_mean_accuracy}, loss - {current_mean_loss}")   
    
    list_of_local_parameter.clear()
    list_of_local_dataset_size.clear()
    list_of_local_accuracy.clear()
    list_of_local_loss.clear()
    
print("\n\n▶▶▶ Round is over.")

total client : 100 , selected client : 10
-- prameter shape --
(5, 5, 1, 32)
(32,)
(5, 5, 32, 64)
(64,)
(3136, 512)
(512,)
(512, 10)
(10,)

▶ Round 1 ◀


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


  evaluation mean : accuracy - 0.9805000424385071, loss - 0.060434915125370026

▶ Round 2 ◀
  evaluation mean : accuracy - 0.989666759967804, loss - 0.03435930237174034

▶ Round 3 ◀
  evaluation mean : accuracy - 0.9958332777023315, loss - 0.016379499807953835

▶ Round 4 ◀
  evaluation mean : accuracy - 0.9985000491142273, loss - 0.007019065320491791

▶ Round 5 ◀
  evaluation mean : accuracy - 0.9971667528152466, loss - 0.009184439666569233

▶ Round 6 ◀
  evaluation mean : accuracy - 0.9978333711624146, loss - 0.00683864438906312

▶ Round 7 ◀
  evaluation mean : accuracy - 0.9994999766349792, loss - 0.0020491222385317087

▶ Round 8 ◀
  evaluation mean : accuracy - 0.9986666440963745, loss - 0.005249458365142345

▶ Round 9 ◀
  evaluation mean : accuracy - 0.9985000491142273, loss - 0.005263152532279491

▶ Round 10 ◀
  evaluation mean : accuracy - 0.9986666440963745, loss - 0.007046651095151901

▶ Round 11 ◀
  evaluation mean : accuracy - 0.999000072479248, loss - 0.0034405719488859177

